# BootCamp 2019 - Week 5

## Heterogeneous Agents Model (Aiyagari 1994)

Prof. Tony Smith

Author: Martina Fraschini

#### Calibrate the transition probabilities
We know that $\pi_{11}=0.9$. Consequently we know that $\pi_{12}=1-\pi_{11}=0.1$. We also know that $\bar{u}_1=0.1$. Using the following equation $$$$ $$ \bar{u}_1 = \bar{u}_1 \pi_{11} + (1-\bar{u}_1) \pi_{21}$$ $$$$ and the fact that $\pi_{22}=1-\pi_{21}$, we obtain the following matrix of transition probabilities: $$$$ 
$$\left[ {\begin{array}{cc}
   \pi_{11} & \pi_{12} \\
   \pi_{21} & \pi_{22} \\
  \end{array} } \right] =
  \left[ {\begin{array}{cc}
   0.9 & 0.1 \\
   0.01 & 0.99 \\
  \end{array} } \right]$$

#### Calculate the steady-state equilibrium value of aggregate capital $\bar{k}^*$

In [25]:
# import package
import numpy as np
from matplotlib import pyplot as plt
from scipy.optimize import root

"""
Legend:
k_grid = possible values for k
k_old, k_new = policy function
k_hat = average capital holdings
k_bar = aggregate capital
"""


# define grid of values for capital k and for shock s
N = 200
k_low = 0.001
k_high = 40
k_grid = np.linspace(k_low, k_high, N)
#k_grid = k_grid * np.ones((2,N)) # we have 2 shocks
kp_grid = np.copy(k_grid)
kp_grid = kp_grid * np.ones((2,N))

# define parameters
beta = .96
delta = .06
alpha = .36

u = 0.1
TP = np.array([[.9, .1], [.01, .99]])
param = [alpha, beta, delta, u]
tol = 1e-8
max_iterations = 10000

# define utility function and its derivative
def u(c):
    if c == 0:
        return -999999
    else:
        return np.log(c)

def u1(c):
    if c == 0:
        return -999999
    else:
        return 1. / c

# define rental and wage rate functions
def rr(k_bar, param):
    alpha, beta, delta, u = param
    rrf = alpha * k_bar**(alpha-1) * (1-u)**(1-alpha)
    return rrf

def wr(k_bar, param):
    alpha, beta, delta, u = param
    wrf = (1-alpha) * k_bar**alpha * (1-u)**(-alpha)
    return wrf
    
def cc(k, eps, k_bar, param):
    alpha, beta, delta, u = param
    ccf = rr(k_bar, param) + wr(k_bar, param)*eps + (1-delta)*k
    return ccf

# define Euler equation
def Euler_eq(x,k,eps,tp,k_grid,k_old,k_bar,param):
    expectation1 = u1(cc(x, 0, k_bar, param) - np.interp(x,k_grid,k_old[0,:])) * rr(k_bar, param)
    expectation2 = u1(cc(x, 1, k_bar, param) - np.interp(x,k_grid,k_old[1,:])) * rr(k_bar, param)
    return u1(cc(k,eps,k_bar,param)-x) - beta*(tp[0]*expectation1 + tp[1]*expectation2)

def k_update(k_old, k_grid, k_bar, TP, param):
    k_new = np.zeros_like(k_old)
    
    for i in range(N):
        for eps in range(2):
            x0 = k_old[eps,i]
            myfun = lambda x: Euler_eq(x,k_grid[i],eps,TP[eps,:],k_grid,k_old,k_bar,param)
            opt = root(myfun, x0)
            k_new[eps,i] = opt.x
            if not opt.success == True:
                print(opt)
            assert opt.success == True, 'solver not successfull'
    
    return k_new

# define function that simulate time series for a typical consumer
def simulation(k_grid, kp, pos, eps_old, TP, param):
    alpha, beta, delta, u = param
    # set seed for random number generation
    np.random.seed(0)
    Tper = 100
    k_hist = np.empty(Tper)
    k_hist[0] = k_grid[pos]
    for j in range(1,Tper):
        eps_new = np.random.binomial(1,TP[eps_old,0],1)
        k_hist[j] = kp[eps_new, pos]
        pos = (np.abs(k_grid-k_hist[j])).argmin()
        eps_old = eps_new
        
    k_hat = k_hist.mean()
    
    return k_hat

# iterate until convergence
k_old = np.ones_like(kp_grid) * k_low
k_bar = simulation(k_grid, k_old, 5, 0, TP, param)
k_bar = k_old.mean()

for iteration in range(max_iterations):
    k_new = k_update(k_old, k_grid, k_bar, TP, param)
    
    k_hat = simulation(k_grid, k_new, 5, 0, TP, param)
    #k_hat = k_old.mean()
    
    difference = abs(k_hat - k_bar)
    #difference = np.max(abs(k_new - k_old))
    
    k_old = k_new.copy()
    k_bar = k_hat

    if difference < tol:
        print('Converged after iteration {}'.format(iteration + 1))
        plt.figure()
        plt.plot(k_grid, k_old[0,:], label=r"$\epsilon = 0$")
        plt.plot(k_grid, k_old[1,:], label=r"$\epsilon = 1$")
        plt.plot(k_grid, k_grid, label="45 degree")
        plt.xlabel('k')
        plt.ylabel("k'")
        plt.legend()
        plt.title('Policy function after convergence')
        plt.show()
        
        break

    fjac: array([[-1.]])
     fun: array([-3.80411441e-82])
 message: 'The number of calls to function has reached maxfev = 400.'
    nfev: 400
     qtf: array([6.15518642e-82])
       r: array([-2.07193617e-164])
  status: 2
 success: False
       x: array([7.77750178e+82])


AssertionError: solver not successfull